In [1]:
import pandas as pd
import tensorflow

2021-11-07 21:55:53.281021: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-07 21:55:53.281065: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
df = pd.read_excel("data train.xlsx")
df.head()

,text,sentiment,Unnamed: 2
0,ayo jawab siapa yg baru tau kalau pos indonesi...,3.0,NaN
1,oktober memperingati hari kesaktian pancasila ...,3.0,NaN
2,alasan sederhana mau coba download pospayoffic...,3.0,NaN
3,tercatat sampai dengan semester i- pertumbuhan...,3.0,NaN
4,pospay punya cerita download aplikasinya dan d...,3.0,NaN


In [10]:
df['sentiment'] = df['sentiment'].fillna(value=2)
df['sentiment'] = df['sentiment'].astype(int)

,text,sentiment,Unnamed: 2,negative
0,ayo jawab siapa yg baru tau kalau pos indonesi...,3,NaN,False
1,oktober memperingati hari kesaktian pancasila ...,3,NaN,False
2,alasan sederhana mau coba download pospayoffic...,3,NaN,False
3,tercatat sampai dengan semester i- pertumbuhan...,3,NaN,False
4,pospay punya cerita download aplikasinya dan d...,3,NaN,False
...,...,...,...,...
1829,Caranya gimana Caranya kalian bungkus hadiah k...,2,NaN,False
1830,Citacita Objektivitas Satu Data Indonesia,2,NaN,False
1831,Dalam acara itu Direktur Utama Pos Indonesia F...,3,NaN,False
1832,Dalam rangka menyambut Maulid Nabi H dan menel...,3,NaN,False


In [11]:
neg = df['sentiment'] == 1
neg.shape

(1834,)